In [23]:
import torch
from torch.nn import functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, set_seed

In [24]:
MODEL_NAME = "flax-community/papuGaPT2"
DEVICE = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
SEED = 42

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

In [25]:
def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label
    
def sentence_prob(sentence_txt: str) -> float:
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(DEVICE)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()  

In [33]:
text = (
    "wprost|wyprosty|wyprostu|wyprost "
    "uwielbiała|wielbił|wielbiła|uwielbił|wielbiło|uwielbiał|uwielbiało|uwielbiały "
    "słuchać|osłuchać|słychać|usłuchać "
    "o|i|e|a|ó|ę|y|ą|u "
    "wartościach własnych|owłosionych macierzy|mocarz|macierzą|macierze|mocarza|mocarze|mocarzy|macierz"
)

groups = [g.split("|") for g in text.split()]
print("Grupy wariantów:")
print(groups)

BEAM_SIZE = 3
beam = [ ("", 0.0) ]   

def extend_beam(beam, words):
    new_beam = []

    for sentence, score in beam:
        for w in words:
            new_sentence = (sentence + " " + w).strip()
            new_score = sentence_prob(new_sentence)# + score
            new_beam.append((new_sentence, new_score))

    new_beam.sort(key=lambda x: x[1], reverse=True)
    return new_beam[:BEAM_SIZE]

for group in groups:
    beam = extend_beam(beam, group)

best_sentence, best_score = beam[0]

for i, b in enumerate(beam):
    print(f"{i + 1}: {b[0]} \t | \t score: {b[1]:.2f}")

print("\nNajlepsze zdanie:")
print(best_sentence)
print("\nScore:", best_score)

Grupy wariantów:
[['wprost', 'wyprosty', 'wyprostu', 'wyprost'], ['uwielbiała', 'wielbił', 'wielbiła', 'uwielbił', 'wielbiło', 'uwielbiał', 'uwielbiało', 'uwielbiały'], ['słuchać', 'osłuchać', 'słychać', 'usłuchać'], ['o', 'i', 'e', 'a', 'ó', 'ę', 'y', 'ą', 'u'], ['wartościach'], ['własnych', 'owłosionych'], ['macierzy', 'mocarz', 'macierzą', 'macierze', 'mocarza', 'mocarze', 'mocarzy', 'macierz']]
1: wprost uwielbiał słuchać o wartościach własnych macierzy 	 | 	 score: -54.64
2: wprost uwielbiały słuchać o wartościach własnych macierzy 	 | 	 score: -55.97
3: wprost uwielbiał słuchać o wartościach własnych mocarzy 	 | 	 score: -58.46

Najlepsze zdanie:
wprost uwielbiał słuchać o wartościach własnych macierzy

Score: -54.635162
